In [ ]:
import math

def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""
    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)


In [ ]:
def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning."""
    """Search all the way to the leaves."""
    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -math.inf, math.inf)

In [ ]:
class Game:
    def __init__(self):
        pass

    def is_terminal(self, state):
        raise NotImplementedError

    def utility(self, state, player):
        raise NotImplementedError

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, action):
        raise NotImplementedError


class Board:
    def __init__(self, height, width, to_move, utility):
        self.height = height
        self.width = width
        self.to_move = to_move
        self.utility = utility
        self.squares = {(x, y): None for x in range(width) for y in range(height)}

    def new(self, square, to_move):
        new_board = Board(self.height, self.width, to_move, self.utility)
        new_board.squares = self.squares.copy()
        new_board.squares[square] = self.to_move
        return new_board


class TicTacToe(Game):
    def __init__(self, height=3, width=3, k=3):
        self.k = k  # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        # Legal moves are any square not yet taken.
        return self.squares - set(square for square in board.squares if board.squares[square] is not None)

    def result(self, board, square):
        # Place a marker for current player on square.
        player = board.to_move
        board = board.new(square, 'O' if player == 'X' else 'X')
        win = self.k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        # Return the value to player; 1 for win, -1 for loss, 0 otherwise.
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        # A board is a terminal state if it is won or there are no empty squares.
        return board.utility != 0 or len(self.squares) == len([square for square in board.squares if board.squares[square] is not None])

    def display(self, board):
        print(board)

    def k_in_row(self, board, player, square, k):
        # True if player has k pieces in a line through square.
        def in_row(x, y, dx, dy):
            return 0 if board.squares[(x, y)] != player else 1 + in_row(x + dx, y + dy, dx, dy)
        return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
                   for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))


# Example usage:
game = TicTacToe()
board = game.initial
while True:
    print("Current board:")
    game.display(board)
    square = input("Enter a square (e.g. (0, 0)): ")
    square = eval(square)
    if square in game.actions(board):
        board = game.result(board, square)
        if game.is_terminal(board):
            print("Game over!")
            break
    else:
        print("Invalid move, try again!")

Current board:


In [ ]:
from collections import defaultdict

class Board(defaultdict):
    """
    A board has the player to move, a cached utility value,
    and a dict of (x, y): player entries, where player is X or O.
    """
    empty = '.'
    off = '#'

    def __init__(self, width=8, height=8, to_move=None, **kwds):
        """
        Initialize the board with the given width, height, and player to move.
        """
        self.width = width
        self.height = height
        self.to_move = to_move
        self.update(kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        """
        Given a dict of (x, y): contents changes, return a new Board with the changes.
        """
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def missing(self, loc):
        """
        Return the contents of the given location, or empty if it's on the board,
        or off if it's off the board.
        """
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.get((x, y), self.empty)
        else:
            return self.off

    def __hash__(self):
        """
        Return a hash value for the board.
        """
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def __repr__(self):
        """
        Return a string representation of the board.
        """
        def row(y):
            return ''.join(self.get((x, y), self.empty) for x in range(self.width))

        return '\n'.join(map(row, range(self.height))) + '\n'

In [ ]:
import random

def random_player(game, state):
    """
    A player that chooses a random action from the available actions.
    """
    return random.choice(list(game.actions(state)))

def player(search_algorithm):
    """
    A game player who uses the specified search algorithm.
    """
    return lambda game, state: search_algorithm(game, state)[1]